In [ ]:
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

import os
import threading
import uuid

crawling_canvas(driver)
crawling_running(driver)
crawling_sneakers(driver)

driver.close()

In [ ]:
from urllib.request import urlretrieve

# 저장 함수
def save_image(results):
    for result in results:
        if not os.path.exists(f"/Volumes/SeaGate/Mac/data-science/Shoes/{result['brand']}-{result['category']}"):
            os.mkdir(f"/Volumes/SeaGate/Mac/data-science/Shoes/{result['brand']}-{result['category']}")

        for idx, src in enumerate(result['img_src']):
            target = f"/Volumes/SeaGate/Mac/data-science/Shoes/{result['brand']}-{result['category']}/{result['name']}_{idx}.jpg"
            urlretrieve(src, target)

In [ ]:
def crawling_canvas(driver):

    url = 'https://search.musinsa.com/category/018'
    
    driver = webdriver.Chrome()
    driver.get(url)
    
    canvas_data = []
    
    canvas_btn = driver.find_element_by_xpath('//*[@id="ui-id-16"]/ul[1]/li[1]/a')
    canvas_btn.click()
    
    driver.implicitly_wait(2)

    ### 브랜드 체크
    brand_len = len(driver.find_elements_by_xpath('//*[@id="best_brand_list"]/dl/dd/ul/li'))
    for i in range(1, brand_len):
        brand = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="best_brand_list"]/dl/dd/ul/li[{i}]')))
        if int(brand.get_attribute('data-count')) > 100:
            brand.click()
            
            driver.implicitly_wait(1)
        
    # 크롤링할 총 페이지 갯수 추출
    max_page_cnt = int(driver.find_element_by_xpath('//*[@id="goods_list"]/div[2]/div[4]/span/span[1]').text)
    # 현재 페이지
    count = 1
    # 앵커 갯수
    now_total_page = len(driver.find_elements_by_xpath('//*[@id="goods_list"]/div[2]/div[1]/div/div/a'))
    
    while True:
        # 크롤링을 위한 작업들
        # 현재 페이지의 상품 갯수를 가져온다.
        max_img_cnt = len(driver.find_elements_by_name('goods_link'))

        # selenium 객체는 인덱스가 1부터 시작.
        # 1부터 마지막까지 크롤링해준다.
        for img_idx in range(1, max_img_cnt, 2):
            # i번째 이미지 객체를 가져온다.
            shoe_img = driver.find_elements_by_name('goods_link')[img_idx]

            # 이미지 객체를 클릭해서 이미지 상세페이지로 접근한다.
            shoe_img.send_keys(Keys.ENTER)

            # 상세페이지 내 이미지 갯수를 가져온다.
            img_cnt = len(driver.find_elements_by_xpath('//*[@id="detail_thumb"]/ul/li'))

            # 이미지 주소를 넣을 배열
            img_src = []
            # 이미지의 브랜드 명
            img_brand = ""
            
            # 상세페이지 내 이미지 갯수만큼 반복문을 돌며
            for cnt in range(1, img_cnt):
                # 선택된 항목이 이미지가 아닐 경우 반복문 종료.
                if len(driver.find_elements_by_xpath(f'//*[@id="thum_{cnt}"]')) == 0:
                    break
                # 이미지는 클릭한다.
                else:
                    detail_img = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="thum_{cnt}"]')))
#                     detail_img = driver.find_element_by_xpath(f'//*[@id="thum_{cnt}"]')
                    detail_img.click()

                    # 클릭한 후, 큰 이미지의 주소를 img_src에 담는다.
                    img_src.append(driver.find_element_by_xpath('//*[@id="bigimg"]').get_attribute('src'))      
            
            try:
                product_parent = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]')))
                product_id = product_parent.find_element_by_tag_name('strong').text
                product_id = product_id.split('/')[1].strip()
            except:
                product_id = uuid.uuid1()
            
            # 상세페이지에서 브랜드를 구한다. 브랜드명에 있는 괄호를 제거한다.
            img_brand = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="page_product_detail"]/div[4]/div[3]/div[2]/p/a[3]')))
            img_brand = img_brand.text.replace('(', '')
            img_brand = img_brand.replace(')', '')
            
            # canvas_data에 json 형식으로 넣어준다.
            canvas_data.append({
                'name': product_id,
                'brand': img_brand,
                'category': "Canvas",
                'img_src': img_src
            })
            
            if len(canvas_data) >= 10:
                # 현재 페이지에서 모은 이미지들을 저장한다.
                save_image(canvas_data)
                # 데이터들을 담았던 배열은 비워준다.
                canvas_data = []

            # 이전페이지(여러 신발을 볼 수 있는 페이지)로 이동한다.
            driver.back()

            driver.implicitly_wait(2)
        
        # 이동을 위한 작업들
        # count(현재위치)가 크롤링할 총 페이지 갯수와 같다면, ( 끝까지 도달했음을 의미 ) 종료!
        if count == max_page_cnt:
            save_image(canvas_data)
            # 데이터들을 담았던 배열은 비워준다.
            canvas_data = []
            break
        
        # 10번째 앵커일 경우 화살표 버튼을 누른다.
        # 화살표 이동시 count += 1
        if count % 10 == 0:
            next_btn = driver.find_elements_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{now_total_page - 1}]')[0]
            next_btn.click()
            count += 1
            
            driver.implicitly_wait(2)
            
            now_total_page = len(driver.find_elements_by_xpath('//*[@id="goods_list"]/div[2]/div[1]/div/div/a'))
        
        # 1 ~ 10 사이의 앵커를오가는 경우.
        # count(현재 위치) + 1 보다 큰 앵커로 이동한다.
        # 이동 시 count += 1.
        else:
            for anchor_idx in range(3, now_total_page - 1):
                next = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{anchor_idx}]').text
                if int(next) == count + 1:
                    next_btn = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{anchor_idx}]')
                    next_btn.click()
                    count += 1
                    break
        
                driver.implicitly_wait(2)

    drive.close()

In [ ]:
def crawling_running(driver):

    url = 'https://search.musinsa.com/category/018'
    
    driver = webdriver.Chrome()
    driver.get(url)

    running_data = []
    
    running_btn = driver.find_element_by_xpath('//*[@id="ui-id-16"]/ul[1]/li[2]/a')
    running_btn.click()
    
    driver.implicitly_wait(2)
    
    brand_len = len(driver.find_elements_by_xpath('//*[@id="best_brand_list"]/dl/dd/ul/li'))
    for i in range(1, brand_len):
        brand = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="best_brand_list"]/dl/dd/ul/li[{i}]')))
        if int(brand.get_attribute('data-count')) > 100:
            brand.click()
            
        driver.implicitly_wait(1)
    
    # 크롤링할 최대 페이지 갯수
    max_page_cnt = int(driver.find_element_by_xpath('//*[@id="goods_list"]/div[2]/div[4]/span/span[1]').text)
    # 현재 페이지 넘버
    count = 1
    # 현재 페이지 기준 앵커의 갯수
    now_total_page = len(driver.find_elements_by_xpath('//*[@id="goods_list"]/div[2]/div[1]/div/div/a'))
    
    while True:
        # 크롤링 작업들
        # 현재 페이지 내 이미지 갯수
        max_img_cnt = len(driver.find_elements_by_name('goods_link'))

        # 이미지 갯수만큼 반복문을 돌면서..
        for i in range(1, max_img_cnt, 2):
            # i번째 신발 객체 가져오기
            shoe_img = driver.find_elements_by_name('goods_link')[i]
            
            # 신발 이미지를 클릭해서, 세부페이지로 접근
            shoe_img.send_keys(Keys.ENTER)

            driver.implicitly_wait(2)
    
            # 세부페이지 내 신발에대한 상세 이미지 갯수 세기
            img_cnt = len(driver.find_elements_by_xpath('//*[@id="detail_thumb"]/ul/li'))

            # 상세이미지 링크를 담을 배열
            img_src = []
            # 신발의 브랜드
            img_brand = ""
            # 신발의 카테고리
            img_category = ""
            # 상세이미지 갯수만큼 반복문을 돌면서..
            for i in range(1, img_cnt):
                # 선택한 객체가 이미지가 아닌경우 반복문 중단
                if len(driver.find_elements_by_xpath(f'//*[@id="thum_{i}"]')) == 0:
                    break
                else:
                    # 이미지 가져오고 클릭 ( 상세이미지를 클릭하면, 이미지가 커진다 )
                    detail_img.WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, 'f//*[@id="thum_{i}"]')))
                    detail_img = driver.find_element_by_xpath(f'//*[@id="thum_{i}"]')
                    detail_img.click()

                    # 커진 상세이미지 주소를 이미지 주소 배열에 넣는다.
                    img_src.append(driver.find_element_by_xpath('//*[@id="bigimg"]').get_attribute('src'))
            
            try:
                product_parent = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]')))
                product_id = product_parent.find_element_by_tag_name('strong').text
                product_id = product_id.split('/')[1].strip()
            except:
                product_id = uuid.uuid1()
              
            
            # 브랜드를 가져오고 정제
            img_brand = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="page_product_detail"]/div[4]/div[3]/div[2]/p/a[3]')))
            img_brand = img_brand.text.replace('(', '')
            img_brand = img_brand.replace(')', '')

            # running_data배열에 신발에대한 정보를 json으로 담는다
            running_data.append({
                'name': product_id,
                'brand': img_brand,
                'category': "Running",
                'img_src': img_src
            })

            # 뒤로 이동
            driver.back()

            driver.implicitly_wait(2)
            
            if len(running_data) >= 10:
                # 현재 페이지에서 모은 사진들을 폴더에 저장한다.
                save_image(running_data)
                # 데이터를 담은 배열은 비워준다.
                running_data = []
        
        # 이동을 위한 작업들
        # 현재위치와 최종 크롤링 페이지위치가 같으면 종료한다.
        if count == max_page_cnt:
            save_image(running_data)
            # 데이터를 담은 배열은 비워준다.
            running_data = []
            break
        
        # 10번째 위치에 있으면 우측화살표를 누른다.
        if count % 10 == 0:
            next_btn = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{now_total_page - 1}]')
            next_btn.click()
            count += 1
            
            driver.implicitly_wait(2)
            # 업데이트된 현재 페이지의 앵커 수를 파악한다.
            now_total_page = len(driver.find_elements_by_xpath('//*[@id="goods_list"]/div[2]/div[1]/div/div/a'))
        
        # 1부터 10사이에서 이동하는 경우, 현재위치+1 지점을 누르자.
        else:
            # 3부터 10까지의 
            for anchor_idx in range(3, now_total_page - 1):
                next = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{anchor_idx}]').text
                if int(next) == count + 1:
                    next_btn = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{anchor_idx}]')
                    next_btn.click()
                    count += 1
                    break
        
                driver.implicitly_wait(2)
    drive.close()

In [ ]:
def crawling_sneakers(driver):

    url = 'https://search.musinsa.com/category/018'
        
    driver = webdriver.Chrome()
    driver.get(url)

    sneakers_data = []
    
    sneakers_btn = driver.find_element_by_xpath('//*[@id="ui-id-16"]/ul[2]/li[2]/a')
    sneakers_btn.click()
    
    driver.implicitly_wait(2)
        
    while True:
        # 크롤링 과정
        # 기타 스니커즈 상위 10개 브랜드를 체크한다.
        # 최대 5개까지 체크가능하기 때문에, 페이지당 5개 체크 -> 크롤링 -> 나머지 5개 체크 -> 크롤링 순으로 진행한다.
        for i in range(2):
            for j in range(i * 5 + 1, i * 5 + 5 + 1):
                # 브렌드 객체를 가져와서 클릭
                brand = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="brand_list_area"]/li[{j}]')))
                brand.click()

                driver.implicitly_wait(1)

            # 크롤링할 최종 페이지 갯수를 구한다.
            max_page_cnt = int(driver.find_element_by_xpath('//*[@id="goods_list"]/div[2]/div[4]/span/span[1]').text)
            # 현재 페이지 위치
            count = 1
            # 현재 페이지 시점에서 이동할 페이지 갯수
            now_total_page = len(driver.find_elements_by_xpath('//*[@id="goods_list"]/div[2]/div[1]/div/div/a'))
            
            # 현재 페이지 내 총 이미지 갯수를 가져온다.
            max_img_cnt = len(driver.find_elements_by_name('goods_link'))

            # 현재 페이지 내 모든 이미지들을 반복문을 통해 순회한다.
            for img_idx in range(1, 10, 2):
                # img_idx번째 이미지 객체를 가져온다.
                shoe_img = driver.find_elements_by_name('goods_link')[img_idx]

                # 신발이미지객체를 클릭해서 세부 페이지로 이동
                shoe_img.send_keys(Keys.ENTER)

                time.sleep(2)

                # 세부 페이지 내 상세이미지 갯수를 구한다.
                img_cnt = len(driver.find_elements_by_xpath('//*[@id="detail_thumb"]/ul/li'))

                # 상세이미지 주소를 담을 배열
                img_src = []

                # 상세이미지 갯수만큼 반복문..
                for cnt in range(1, img_cnt):
                    # 선택한 객체가 이미지가 아니라면 반복문 종료
                    if len(driver.find_elements_by_xpath(f'//*[@id="thum_{cnt}"]')) == 0:
                        break
                    else:
                        # 상세이미지를 클릭해서, 이미지를 크게만든다.
                        detail_img = WebDriverWait(driver, 5).until(EC.precense_of_element_located((By.XPATH, f'//*[@id="thum_{cnt}"]')))
                        detail_img.click()

                        # 이미지의 주소를 담는다.
                        img_src.append(driver.find_element_by_xpath('//*[@id="bigimg"]').get_attribute('src'))
                
                try:
                    product_parent = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]')))
                    product_id = product_parent.find_element_by_tag_name('strong').text
                    product_id = product_id.split('/')[1].strip()
                except:
                    product_id = uuid.uuid1()
              

                # 기타 스니커즈 정보를 json 형태로 담는다.
                sneakers_data.append({
                    'name': product_id,
                    'brand': 'Other',
                    'category': "Sneakers",
                    'img_src': img_src
                })

                driver.back()

                time.sleep(2)

                if len(sneakers_data) >= 10:
                    # 현재 페이지에서 모은 사진들을 폴더에 저장한다.
                    save_image(sneakers_data)
                    # 데이터를 담은 배열은 비워준다.
                    sneakers_data = []
            
            
            del_btn = driver.find_element_by_class_name('delete-btn-red')
            del_btn.click()

        # 크롤링 후 페이지 이동
        # 현재 위치가 크롤링할 페이지의 종점이면 반복문을 종료.
        if count == max_page_cnt:
            save_image(sneakers_data)
            # 데이터를 담은 배열은 비워준다.
            sneakers_data = []
            break

            # 현재 위치가 10번째 페이지면, 화살표를 누른다.
        if count % 10 == 0:
            next_btn = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{now_total_page - 1}]')
            next_btn.click()
            count += 1

            time.sleep(2)

            now_total_page = len(driver.find_elements_by_xpath('//*[@id="goods_list"]/div[2]/div[1]/div/div/a'))

        # 3 ~ 10까지 돌면서, count보다 1 큰 위치 ( 다음위치 )로 이동한다.
        else:
            for anchor_idx in range(3, now_total_page - 1):
                next = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{anchor_idx}]').text
                if int(next) == count + 1:
                    next_btn = driver.find_element_by_xpath(f'//*[@id="goods_list"]/div[2]/div[1]/div/div/a[{anchor_idx}]')
                    next_btn.click()
                    count += 1
                    break

                time.sleep(2)
    drive.close()